# Neural Machine Translation

**Reference:** Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." In Advances in neural information processing systems, pp. 3104-3112. 2014. ([Paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks), [Sample code](https://github.com/tensorflow/nmt))

This Assignment has been done by

<h2> Abdurrahman Beyaz</h2>
00684383 

[Email](aalabrash18@ku.edu.tr) 

<h2> Ahmed Masry </h2>
0061868

[Email](amasry17@ku.edu.tr )


[To Download model s2s_v1.jld2](https://drive.google.com/uc?export=download&confirm=iXgT&id=1ztqkspEia4e3yE9VYfyS5olAwKgH33On) BLEU ~3

[To Download model s2s_v1.1.jld2](https://drive.google.com/uc?id=1b9YXxrMeZqHayPhlXyXg5_f-6CKR0RYb&export=download) BLEU ~3

[s2s_v2_f_4.jld2]( https://drive.google.com/open?id=1KK25lQ9vbBdavi9nnle2FxF72dlu47yC) BLEU ~8

[s2s_v2.jld](https://drive.google.com/file/d/1Sj8IAMvgCApB85UC4w46SibBTsrQTZ-S/view?usp=sharing) hightest BLEU 8.04

In [1]:
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

@size (macro with 1 method)

## Part -1. Types from the last project

Please copy the following types and related functions from the last project: `Vocab`,
`TextReader`, `Embed`, `Linear`, `mask!`.

In [2]:
struct Vocab
    w2i::Dict{String,Int}
    i2w::Vector{String}
    unk::Int
    eos::Int
    tokenizer
end

function Vocab(file::String; tokenizer=split, vocabsize=Inf, mincount=1, unk="<unk>", eos="<s>")
    word_count = Dict{String,Int}()
    w2i = Dict{String,Int}()
    i2w = Vector{String}()
    int_unk = get!(w2i, unk, 1+length(w2i))
    int_eos = get!(w2i, eos, 1+length(w2i))
    for line in eachline(file)
        line = tokenizer(line)
        for word in line
            if haskey(word_count, word)
                word_count[word] += 1
            else
                word_count[word] = 1
            end
        end
    end
    word_count = collect(word_count)
    sort!(word_count, rev=true, by=x->x[2])
    # constructing w2i
    for pair in word_count
        if pair[2] >= mincount
            get!(w2i, pair[1], 1+length(w2i))
            if length(w2i) >= vocabsize
                break
            end
        end
    end
    w2i_array = collect(w2i)
    sort!(w2i_array, by=x->x[2])
    for pair in w2i_array
        push!(i2w, pair[1])
    end
    return Vocab(w2i, i2w, int_unk, int_eos, tokenizer)
end

Vocab

In [3]:
struct TextReader
    file::String
    vocab::Vocab
end

function Base.iterate(r::TextReader, s=nothing)
    # Your code here
    s ==nothing ? file = open(r.file) : file =s
    
    if eof(file) == true
        close(file)
        return nothing
    end
    line = readline(file)
    text = r.vocab.tokenizer(line)
    arr = [get(r.vocab.w2i,word,r.vocab.w2i["<unk>"]) for word in text ]
    return (arr, file)
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

struct Embed; w; end

function Embed(vocabsize::Int, embedsize::Int)
    # Your code here
    Embed(param(embedsize, vocabsize, atype = KnetArray{Float32}))
end

function (l::Embed)(x)
    # Your code here
    l.w[:,x]
end

struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    # Your code here
    Linear(param(outputsize, inputsize, atype = KnetArray{Float32}), param0(outputsize, atype = KnetArray{Float32}))
end

function (l::Linear)(x)
    # Your code here
    l.w*x .+ l.b
end

function mask!(a,pad)
    # Your code here
    #b = deepcopy(a)
    for k in 1:size(a, 1)
        if a[k , size(a, 2)]!= pad
            continue
        end
        
        indices = []
        for i in 1:size(a[k, :], 1)
            if a[k, i] == pad
                push!(indices, i)
            end
        end
        indices = reverse(indices)
        for j in 1:size(indices, 1)-1
            if indices[j] == indices[j+1] + 1
                a[k, indices[j]] = 0
            else
                break
            end
        end
    end
    a
end

mask! (generic function with 1 method)

## Part 0. Load data

We will use the Turkish-English pair from the [TED Talks Dataset](https://github.com/neulab/word-embeddings-for-nmt) for our experiments.

In [4]:

datadir = "datasets/tr_to_en"

if !isdir(datadir)
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount=5)
    en_vocab = Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[4]:18


Test Passed

## Part 1. Minibatching

For minibatching we are going to design a new iterator: `MTData`. This iterator is built
on top of two TextReaders `src` and `tgt` that produce parallel sentences for source and
target languages.

In [5]:
struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader, tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

### iterate(::MTData)

Define the `iterate` function for the `MTData` iterator. `iterate` should return a
`(batch, state)` pair or `nothing` if there are no more batches.  The `batch` is a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. The `state` is a pair of `(src_state,tgt_state)` which can be used
to iterate `d.src` and `d.tgt` to get more sentences.  `iterate(d)` without a second
argument should initialize `d` by emptying its buckets and calling `iterate` on the inner
iterators `d.src` and `d.tgt` without a state. Please review the documentation on
iterators from the last project.

To keep similar length sentences together `MTData` uses arrays of similar length sentence
pairs called buckets.  Specifically, the `(src_sentence,tgt_sentence)` pairs coming from
`src` and `tgt` are pushed into `d.buckets[i]` when the length of the source sentence is
in the range `((i-1)*d.bucketwidth+1):(i*d.bucketwidth)`. When one of the buckets reaches
`d.batchsize` `d.batchmaker` is called with the full bucket producing a 2-D batch, the
bucket is emptied and the batch is returned. If `src` and `tgt` are exhausted the
remaining partially full buckets are turned into batches and returned in any order. If the
source sentence length is larger than `length(d.buckets)*d.bucketwidth`, the last bucket
is used.

Sentences above a certain length can be skipped using the `d.maxlength` field, and
transposed `x,y` arrays can be produced using the `d.batchmajor` field.

In [6]:
function Base.iterate(d::MTData, state=nothing)
    # Your code here
    
    batch = nothing
    state = state
    if state == nothing
        for i in 1:length(d.buckets)
           d.buckets[i] = [] 
        end
    end
    
    while batch == nothing
        #print("W")
        if state == "index"
            #println("Hello. It's me.")
            found = false
            for i in 1:length(d.buckets)
                buck = d.buckets[i]
                if length(buck) != 0
                    #println(length(buck))
                    found = true
                    batch = d.batchmaker(d, buck) 
                    d.buckets[i] = []
                    break
                end
            end
            
            if found
                return (batch, "index") 
            else
                #print("Nothing")
                return nothing
            end
        end
        
        state_src, state_tgt = state==nothing ? (nothing, nothing) : state
        src_pair = iterate(d.src, state_src)
        tgt_pair = iterate(d.tgt, state_tgt)
        
        if src_pair == nothing
            #println("Index reached")
            state = "index"
            continue
        end

        src_sentence, state_src = src_pair
        tgt_sentence, state_tgt = tgt_pair
        
        if length(src_sentence) > d.maxlength
            state = (state_src, state_tgt)
            continue
        end
        if length(src_sentence) > length(d.buckets)*d.bucketwidth
            last_bucket = d.buckets[length(d.buckets)]
            push!(last_bucket, (src_sentence, tgt_sentence))
            if length(last_bucket) == d.batch_size
               batch = d.batchmaker(d, last_bucket)
               d.buckets[length(d.buckets)] = []
            end
        else 
            for i in 1:length(d.buckets)
               if (length(src_sentence) >= ((i-1)*d.bucketwidth+1)) & (length(src_sentence) <= (i*d.bucketwidth))
                   push!(d.buckets[i], (src_sentence, tgt_sentence))
                    if length(d.buckets[i]) == d.batchsize
                        #println("I am here")
                        batch = d.batchmaker(d, d.buckets[i])
                        d.buckets[i] = []
                    end
                    break
                end
            end   
        end
        state = (state_src, state_tgt)
    end
    #print("Bye")
    return (batch, state)
end

### arraybatch

Define `arraybatch(d, bucket)` to be used as the default `d.batchmaker`. `arraybatch`
takes an `MTData` object and an array of sentence pairs `bucket` and returns a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. Note that the sentences in the bucket do not have any `eos` tokens
and they may have different lengths. `arraybatch` should copy the source sentences into
`x` padding shorter ones on the left with `eos` tokens. It should copy the target
sentences into `y` with an `eos` token in the beginning and end of each sentence and
shorter sentences padded on the right with extra `eos` tokens.

In [7]:
function arraybatch(d::MTData, bucket)
    # Your code here
    #println("Bucket: ", length(bucket), ", ", length(bucket[1][1]), ", ", length(bucket[1][2]))
    srclength = 0
    tgtlength = 0
    
    x = []
    y = []
    
    for pair in bucket
        src_sentence, tgt_sentence = pair
        if length(src_sentence) > srclength
           srclength = length(src_sentence)
        end
        if length(tgt_sentence) > tgtlength
           tgtlength = length(tgt_sentence)
        end
    end
    
    for pair in bucket
        src_sentence, tgt_sentence = pair
        src_sen = []
        tgt_sen = []
                
        #Target part. 
        tgt_eos = d.tgt.vocab.eos
        push!(tgt_sen, tgt_eos)
        for w in tgt_sentence
            push!(tgt_sen, w) 
        end
        while length(tgt_sen) != (tgtlength + 2)
            push!(tgt_sen, tgt_eos)
        end
        
        #Source part. 
        src_eos = d.src.vocab.eos
        eos_num = srclength - length(src_sentence)
        i = 0
        while i!= eos_num
            push!(src_sen, src_eos)
            i += 1
        end
        for w in src_sentence
            push!(src_sen, w) 
        end        
        push!(x, src_sen)
        push!(y, tgt_sen)
    end
    #println(size(hcat(x...)))
    #println(length(hcat(x...)))
    #println(length(hcat(x...)[1]))
    return Array(transpose(hcat(x...))), Array(transpose(hcat(y...)))
end

arraybatch (generic function with 1 method)

In [8]:
@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x,y = first(dtst)
@test length(collect(dtst)) == 48
@test size.((x,y)) == ((128,10),(128,24))
@test x[1,1] == tr_vocab.eos
@test x[1,end] != tr_vocab.eos
@test y[1,1] == en_vocab.eos
@test y[1,2] != en_vocab.eos
@test y[1,end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[8]:1


Test Passed

## Part 2. Sequence to sequence model without attention

In this part we will define a simple sequence to sequence encoder-decoder model for
machine translation.

In [9]:
struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

### S2S_v1 constructor

Define the S2S_v1 constructor using your predefined layer types (Embed, Linear), and the
Knet RNN type. Please review the RNN documentation using `@doc RNN`, paying attention to
the following options in particular: `numLayers`, `bidirectional`, `dropout`, `dataType`,
`usegpu`. The last two are important if you experiment with array types other than the
default `KnetArray{Float32}`: make sure the RNNs use the same array type as the other
layers. Note that if the encoder is bidirectional, its `numLayers` should be half of the
decoder so that their hidden states match in size.

In [10]:
function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int,       # embedding size for target language
                srcvocab::Vocab,     # vocabulary for source language
                tgtvocab::Vocab;     # vocabulary for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    # Your code here
    src_embedd_layer = Embed(length(srcvocab.i2w), srcembsz)
    tgt_embedd_layer = Embed(length(tgtvocab.i2w), tgtembsz)
    proj = Linear(hidden, length(tgtvocab.i2w))
    encoder_layers = layers
    if bidirectional 
        encoder_layers /= 2
    end
    
    encoder = RNN(srcembsz, hidden, numLayers = encoder_layers, bidirectional = bidirectional, dropout = dropout)
    decoder = RNN(tgtembsz, hidden, numLayers = layers, dropout = dropout)
    S2S_v1(src_embedd_layer, encoder, tgt_embedd_layer, decoder, proj, dropout, srcvocab, tgtvocab)
end

S2S_v1

### S2S_v1 loss function

Define the S2S_v1 loss function that takes `src`, a source language minibatch, and `tgt`,
a target language minibatch and returns either a `(total_loss, num_words)` pair if
`average=false`, or `(total_loss/num_words)` average if `average=true`.

Assume that `src` and `tgt` are integer arrays of size `(B,Tx)` and `(B,Ty)` respectively,
where `B` is the batch size, `Tx` is the length of the longest source sequence, `Ty` is
the length of the longest target sequence. The `src` sequences only contain words, the
`tgt` sequences surround the words with `eos` tokens at the start and end. This allows
columns `tgt[:,1:end-1]` to be used as the decoder input and `tgt[:,2:end]` as the desired
decoder output.

Assume any shorter sentences in the batches have been padded with extra `eos` tokens on
the left for `src` and on the right for `tgt`. Don't worry about masking `src` for the
encoder, it doesn't have a significant effect on the loss. However do mask `tgt` before
`nll`: you do not want the padding tokens to be counted in the loss calculation.

Please review `@doc RNN`: in particular the `r.c` and `r.h` fields can be used to get/set
the cell and hidden arrays of an RNN (note that `0` and `nothing` act as special values).

RNNs take a dropout value at construction and apply dropout to the input of every layer if
it is non-zero. You need to handle dropout for other layers in the loss function or in
layer definitions as necessary.

In [11]:
function (s::S2S_v1)(src, tgt; average=true)
    s.encoder.c, s.encoder.h = 0, 0
    src_embed_out = s.srcembed(src) #;@show typeof(src_embed_out),size(src_embed_out)
    s.encoder(src_embed_out) #;@show s.encoder
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
    tgt_embed_out = s.tgtembed(tgt[:, 1:end-1]) #; @show size(tgt_embed_out)
    y_de = s.decoder(tgt_embed_out) #; @show size(y_de,1),size(y_de,2),size(y_de,3);
    y_reshaped = reshape(y_de, (size(y_de,1), size(y_de,2)*size(y_de,3))) # ;@show size(y_reshaped)
    scores = s.projection(y_reshaped) #;@show size(scores)
    target=tgt[:, 2:end]
    mask!(target, s.srcvocab.eos)#;    @show size(dec_op)
    target=reshape(target,:)
    nll(scores,target, average=average)
end

In [12]:
@info "Testing S2S_v1"
Knet.seed!(1)
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2)
(x,y) = first(dtst)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
@test all(model(x,y; average=false) .≈ (14097.471f0, 1432))

┌ Info: Testing S2S_v1
└ @ Main In[12]:1


Test Passed

In [13]:
model(x,y)

9.844618f0

### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(x,y)` pairs such as `MTData` and `model(x,y;average)` is a model like
`S2S_v1` that computes loss on a single `(x,y)` pair.

In [3]:
function loss(model, data; average=true)
    loss,insts = 0,0
    loss_sum,loss_total = 0,0
    for (x, y) in data
        loss, insts = model(x, y, average=false)
        loss_total += insts
        loss_sum += loss
    end
    average ? (return loss_sum/loss_total) : (return loss_sum, loss_total)
end

loss (generic function with 1 method)

In [17]:
@info "Testing loss"
@test all(loss(model, dtst, average=false) .≈ (1.0429117f6, 105937))
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0429178f6, 105937) with batchsize==1.

┌ Info: Testing loss
└ @ Main In[17]:1


Test Passed

### Training SGD_v1

The following function can be used to train our model. `trn` is the training data, `dev`
is used to determine the best model, `tst...` can be zero or more small test datasets for
loss reporting. It returns the model that does best on `dev`.

In [17]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

train! (generic function with 1 method)

You should be able to get under 3.40 dev loss with the following settings in 10
epochs. The training speed on a V100 is about 3 mins/epoch or 40K words/sec, K80 is about
6 times slower. Using settings closer to the Luong paper (per-sentence loss rather than
per-word loss, SGD with lr=1, gclip=1 instead of Adam), you can get to 3.17 dev loss in
about 25 epochs. Using dropout and shuffling batches before each epoch significantly
improve the dev loss. You can play around with hyperparameters but I doubt results will
get much better without attention. To verify your training, here is the dev loss I
observed at the beginning of each epoch in one training session:
`[9.83, 4.60, 3.98, 3.69, 3.52, 3.41, 3.35, 3.32, 3.30, 3.31, 3.33]`

In [18]:
@info "Training S2S_v1"
epochs = 10
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trn20 = ctrn[1:20]
dev38 = collect(ddev)
#Uncomment this to train the model (This takes about 30 mins on a V100):
model = train!(model, trnx10, dev38, trn20)
#Uncomment this to save the model:
#Knet.save("s2s_v2.jld2","model",model)
#Uncomment this to load the model:
#model = Knet.load("s2s_v2.jld2","model")

┌ Info: Training S2S_v1
└ @ Main In[18]:1
┣████████████████████┫ [100.00%, 14330/14330, 36:32/36:32, 6.54i/s] (3.481387f0, 2.0574226f0)


S2S_v1(Embed(P(KnetArray{Float32,2}(512,38126))), LSTM(input=512,hidden=512,bidirectional,dropout=0.2), Embed(P(KnetArray{Float32,2}(512,18857))), LSTM(input=512,hidden=512,layers=2,dropout=0.2), Linear(P(KnetArray{Float32,2}(18857,512)), P(KnetArray{Float32,1}(18857))), 0.2, Vocab(Dict("dev" => 1277,"komuta" => 13566,"ellisi" => 25239,"adresini" => 22820,"yüzeyi" => 4051,"paris'te" => 9494,"kafamdaki" => 18790,"yüzeyinde" => 5042,"geçerlidir" => 6612,"kökten" => 7774…), ["<unk>", "<s>", ".", ",", "bir", "ve", "bu", "''", "``", "için"  …  "seçmemiz", "destekleyip", "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde"], 1, 2, split), Vocab(Dict("middle-income" => 13398,"photosynthesis" => 7689,"polarizing" => 17881,"henry" => 4248,"abducted" => 15691,"rises" => 6225,"hampshire" => 13888,"whiz" => 16835,"cost-benefit" => 13137,"progression" => 5549…), ["<unk>", "<s>", ",", ".", "the", "and", "to", "of", "a", "that"  …  "archaea", "handsh

### Generating translations

With a single argument, a `S2S_v1` object should take it as a batch of source sentences
and generate translations for them. After passing `src` through the encoder and copying
its hidden states to the decoder, the decoder is run starting with an initial input of all
`eos` tokens. Highest scoring tokens are appended to the output and used as input for the
subsequent decoder steps.  The decoder should stop generating when all sequences in the
batch have generated `eos` or when `stopfactor * size(src,2)` decoder steps are reached. A
correctly shaped target language batch should be returned.

In [19]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    batch_size = size(src)[1]
    src_length = size(src)[2]
    s.encoder.h, s.encoder.c = 0, 0
    s.encoder(s.srcembed(src))
    s.decoder.h, s.decoder.c = s.encoder.h, s.encoder.c
    dec_input = s.tgtembed(repeat([s.tgtvocab.eos], batch_size))
    trans_sens = []
    eos_cond = repeat([false], batch_size)
    while !all(eos_cond) 
        hidden_state = s.decoder(dec_input)
        scores = s.projection(hidden_state)
        tokens = vec(map(x -> x[1], argmax(scores, dims=1)))
        push!(trans_sens, tokens)
        eos_cond[findall(i->i==s.tgtvocab.eos, tokens)] .= true
        if length(trans_sens) >= stopfactor * size(src,2)
            break
        end
        dec_input = s.tgtembed(tokens)
    end
    #println("\n\n\n")
    return hcat(trans_sens...)
end

In [20]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

int2str (generic function with 1 method)

In [21]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize=1) |> collect
(src,tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src,model.srcvocab))
println("REF: ", int2str(tgt,model.tgtvocab))
println("OUT: ", int2str(out,model.tgtvocab))
# Here is a sample output:
# SRC: çin'e 15 şubat 2006'da ulaştım .
# REF: i made it to china on february 15 , 2006 .
# OUT: i got to china , china , at the last 15 years .

┌ Info: Generating some translations
└ @ Main In[21]:1


SRC: çin'e 15 şubat 2006'da ulaştım .
REF: i made it to china on february 15 , 2006 .
OUT: and in the summer , i went to the moon , and i was in the middle .


### Calculating BLEU

BLEU is the most commonly used metric to measure translation quality. The following should
take a model and some data, generate translations and calculate BLEU.

In [22]:
function bleu(s2s,d::MTData)
    d = MTData(d.src,d.tgt,batchsize=1)
    reffile = d.tgt.file
    hypfile,hyp = mktemp()
    for (x,y) in progress(collect(d))
        g = s2s(x)
        for i in 1:size(y,1)
            println(hyp, int2str(g[i,:], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download("https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl", "multi-bleu.perl")
    run(pipeline(`cat $hypfile`,`perl multi-bleu.perl $reffile`))
    return hypfile
end

bleu (generic function with 1 method)

Calculating dev BLEU takes about 45 secs on a V100. We get about 8.0 BLEU which is pretty
low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
of ~8 is not sufficient to generate meaningful translations.

In [23]:
@info "Calculating BLEU"
bleu(model, ddev)

┌ Info: Calculating BLEU
└ @ Main In[23]:1
┣████████████████████┫ [100.00%, 4045/4045, 01:02/01:02, 64.95i/s] 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LC_CTYPE = "UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


BLEU = 8.04, 37.4/11.9/4.7/2.0 (BP=1.000, ratio=1.072, hyp_len=88469, ref_len=82502)


It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


"/tmp/jl_evIopI"

To improve the quality of translations we can use more training data, different training
and model parameters, or preprocess the input/output: e.g. splitting Turkish words to make
suffixes look more like English function words may help. Other architectures,
e.g. attention and transformer, perform significantly better than this simple S2S model.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*